# Milestone 3: Memory Integration

This notebook demonstrates:
1. Adding conversation memory with LangGraph
2. Multi-turn conversations
3. Context-aware responses

In [ ]:
import sys
sys.path.insert(0, '..')

from src.rag_baseline import BaselineRAG
from src.memory import ConversationMemory, MemoryRAGAgent, create_memory_agent
from dotenv import load_dotenv

load_dotenv()
print("✓ Modules imported")

## Step 1: Create Memory-Enabled Agent

In [ ]:
# Initialize RAG with memory
rag = BaselineRAG("../faiss_index")
memory = ConversationMemory(max_messages=10)
agent = MemoryRAGAgent(rag, memory)

print("✓ Memory-enabled agent created")

## Step 2: Multi-Turn Conversation

In [ ]:
# Conversation example
response1 = agent.chat("What is GDPR?")
print(f"Turn 1: {response1['answer'][:150]}...\n")

response2 = agent.chat("Can you elaborate on the key principles?")
print(f"Turn 2: {response2['answer'][:150]}...\n")

response3 = agent.chat("What are the penalties for violating it?")
print(f"Turn 3: {response3['answer'][:150]}...\n")

print(f"\nConversation summary: {agent.get_conversation_summary()}")

## Summary

Memory integration enables context-aware conversations.